![(book cover)](https://covers.oreillystatic.com/images/0636920167433/cat.gif "(book cover)")
# Programming Quantum Computers by O'Reilly Media -  [book info](http://shop.oreilly.com/product/0636920167433.do)  - [all code samples](https://oreilly-qc.github.io)

## Code samples for Chapter 8
These code samples were written by Mariia Mykhailova.

### Example 8-1: Using the phase estimation primitive

In [1]:
// Example 8-1: Using the phase estimation primitive

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Characterization;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Oracles;

// Helper operation to define powers of H gate
operation HPower (power : Int, register : Qubit[]) : Unit is Adj + Ctl {
    // We know that H² = I, so we just need to apply H if power is odd
    if (power % 2 == 1) {
        H(register[0]);
    }
}

operation UsingQPE () : Unit {
    let precision = 4;
    // Allocate qubits to hold the eigenstate of H and the phase (in a big endian register)
    using ((eigenstate, phaseRegister) = (Qubit[1], Qubit[precision])) {
        // Prepare the eigenstate of H gate corresponding to eigenphase of 180°
        // (for 0° we'd use 0.25 * PI() as rotation angle)
        Ry(-0.75 * PI(), eigenstate[0]);
        
        // Call library implementation of quantum phase estimation
        let phaseRegisterBE = BigEndian(phaseRegister);
        let powerUnitary = DiscreteOracle(HPower);
        QuantumPhaseEstimation(powerUnitary, eigenstate, phaseRegisterBE);
        
        // Read out the phase
        let phase = IntAsDouble(MeasureInteger(BigEndianAsLittleEndian(phaseRegisterBE))) / IntAsDouble(1 <<< precision);
        Message($"Estimated phase = {phase * 360.0}°");

        ResetAll(eigenstate + phaseRegister);
    }
}

HPower, UsingQPE

In [2]:
%simulate UsingQPE

Estimated phase = 180°


()

### Example 8-2: Implementation of the phase estimation primitive

In [3]:
// Example 8-2: Implementation of the phase estimation primitive

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Characterization;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Oracles;

operation QPE (powerUnitary : ((Int, Qubit[]) => Unit is Adj+Ctl), 
               eigenstate : Qubit[],
               phaseRegister : Qubit[]) : Unit {
    ApplyToEach(H, phaseRegister);
    
    // Apply conditional powers of U
    for (i in 0 .. Length(phaseRegister) - 1) {
        Controlled powerUnitary([phaseRegister[i]], (1 <<< i, eigenstate));
    }
    
    QFTLE(LittleEndian(phaseRegister));
}

// Helper operation to define powers of the rotation gate
operation RotatePower (power : Int, register : Qubit[]) : Unit is Adj + Ctl {
    // To apply higher powers of the rotation gate, we can rotate by multiples of the angle
    R1(-PI() * 5.0/6.0 * IntAsDouble(power), register[0]);
}

operation ImplementingQPE () : Unit {
    let precision = 3;
    // Allocate qubits to hold the eigenstate of H and the phase (in a big endian register)
    using ((eigenstate, phaseRegister) = (Qubit[1], Qubit[precision])) {
        // Prepare the eigenstate of the rotation gate corresponding to eigenphase of 150°;
        // for R1 gate, that is simply a |1⟩
        X(eigenstate[0]);
        
        // Call our implementation of quantum phase estimation
        QPE(RotatePower, eigenstate, phaseRegister);
        
        // Inspect the state we obtain after applying QPE
        DumpRegister((), phaseRegister);
        
        // Read out the phase
        let phase = IntAsDouble(MeasureInteger(LittleEndian(phaseRegister))) / IntAsDouble(1 <<< precision);
        Message($"Estimated phase = {phase * 360.0}°");

        ResetAll(eigenstate + phaseRegister);
    }
}

ImplementingQPE, QPE, RotatePower

In [4]:
%simulate ImplementingQPE

|0⟩	0.10825317547305487 + 0.029006350946109552𝑖
|1⟩	0.13528279153744854 + -0.017810317386500216𝑖
|2⟩	0.18749999999999992 + -0.10825317547305492𝑖
|3⟩	0.5048822514075066 + -0.6579753603314554𝑖
|4⟩	-0.10825317547305481 + 0.4040063509461098𝑖
|5⟩	0.023210857516441776 + 0.1763039664403905𝑖
|6⟩	0.06250000000000003 + 0.10825317547305492𝑖
|7⟩	0.08662409953860287 + 0.06646900938534578𝑖

Estimated phase = 90°


()